<a href="https://colab.research.google.com/github/Papa-Panda/random_thoughts/blob/main/DiffusionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Define a simple diffusion model
class DiffusionModel(nn.Module):
    # def __init__(self, channels=3, steps=1000, device='cuda'):
    def __init__(self, channels=3, steps=1000, device='cpu'):
        super(DiffusionModel, self).__init__()
        self.steps = steps
        self.device = device

        self.net = nn.Sequential(
            nn.Conv2d(channels, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, channels, kernel_size=3, stride=1, padding=1)
        )

    def forward(self, x, t):
        for _ in range(self.steps):
            noise = torch.randn_like(x).to(self.device)
            x = x * torch.exp(-torch.sqrt(torch.tensor([1.0], device=self.device) * t)) + noise
            x = self.net(x)
        return x

# Define training function
# def train(model, dataloader, epochs, lr, device='cuda'):
def train(model, dataloader, epochs, lr, device='cpu'):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.to(device)
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, _ in dataloader:
            inputs = inputs.to(device)

            # Generate random time steps
            timesteps = torch.rand(inputs.size(0), device=device)

            optimizer.zero_grad()

            outputs = model(inputs, timesteps)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

# Main function
def main():
    # Hyperparameters
    batch_size = 64
    lr = 1e-4
    epochs = 10

    # Load CIFAR-10 dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Initialize diffusion model
    diffusion_model = DiffusionModel()

    # Train diffusion model
    train(diffusion_model, train_loader, epochs, lr)

if __name__ == "__main__":
    main()


Files already downloaded and verified


RuntimeError: The size of tensor a (32) must match the size of tensor b (64) at non-singleton dimension 3